In [24]:
import subprocess
import os

def export_msinfo_report(output_path=None):
    if output_path is None:
        output_path = os.path.join(os.getcwd(), f"msinfo32_report.xml")

    command = ["msinfo32.exe", "/nfo", output_path]

    try:
        subprocess.run(command, check=True)
        print(f"Отчёт успешно сохранён: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при запуске msinfo32: {e}")

export_msinfo_report()


Отчёт успешно сохранён: d:\miwn36\Python\Coursework\CourseWork-backend\msinfo32_report.xml


In [18]:
import wmi

def get_disk_serial_number():
    """Получает серийный номер жесткого диска с помощью wmi."""
    try:
        c = wmi.WMI()
        # Находим объект диска
        disk = c.query("SELECT SerialNumber FROM Win32_DiskDrive")[0]
        return disk.SerialNumber
    except Exception as e:
        return f"Произошла ошибка: {e}"

if __name__ == "__main__":
    serial_number = get_disk_serial_number()
    print(f"Серийный номер жесткого диска: {serial_number}")


Серийный номер жесткого диска: S3YANB0M383859K


In [6]:
import win32api

def get_monitor_resolution_and_refresh_rate():
    # Получить разрешение и частоту обновления первого монитора
    screen_width = win32api.GetSystemMetrics(0)
    screen_height = win32api.GetSystemMetrics(1)
    
    device = win32api.EnumDisplayDevices(None, 0)
    
    print(f"Разрешение экрана: {screen_width} x {screen_height}")
    print(f"Частота обновления: {win32api.EnumDisplaySettings(device.DeviceName, -1).DisplayFrequency} Гц")

if __name__ == "__main__":
    get_monitor_resolution_and_refresh_rate()


Разрешение экрана: 1920 x 1080
Частота обновления: 60 Гц


In [19]:
import wmi

# Подключаемся к пространству root\wmi
c = wmi.WMI(namespace="root\\wmi")

# Получаем информацию о мониторах
for monitor in c.WmiMonitorID():
    # SerialNumberID возвращает массив чисел (коды символов)
    serial = "".join([chr(i) for i in monitor.SerialNumberID if i > 0])
    name = "".join([chr(i) for i in monitor.UserFriendlyName if i > 0])
    manufacturer = "".join([chr(i) for i in monitor.ManufacturerName if i > 0])

    print(f"Монитор: {name}")
    print(f"Производитель: {manufacturer}")
    print(f"Серийный номер: {serial}\n")


Монитор: LG IPS FULLHD
Производитель: GSM
Серийный номер: 16843009



In [23]:
import wmi
import json

def safe_str(val):
    return str(val).strip() if val else ""

def decode_wmi_string(arr):
    return "".join([chr(i) for i in arr if i > 0]).strip()

c = wmi.WMI()
c_mon = wmi.WMI(namespace="root\\wmi")

data = {}

# --- Жёсткие диски ---
data["disks"] = [
    {
        "model": safe_str(disk.Model),
        "serial": safe_str(disk.SerialNumber),
        "device_id": safe_str(disk.DeviceID)
    }
    for disk in c.Win32_DiskDrive()
]

# --- Мониторы ---
data["monitors"] = [
    {
        "name": decode_wmi_string(m.UserFriendlyName),
        "manufacturer": decode_wmi_string(m.ManufacturerName),
        "serial": decode_wmi_string(m.SerialNumberID)
    }
    for m in c_mon.WmiMonitorID()
]

# --- Тип подключения мониторов ---
connection_map = {
    0: "Встроенный",
    1: "VGA",
    2: "S-Video",
    3: "Composite",
    4: "Component",
    5: "DVI/HDMI (TMDS)",
    6: "HDMI",
    8: "LVDS",
    10: "DisplayPort",
    11: "WirelessDisplay"
}
data["monitor_connections"] = [
    {
        "instance": safe_str(conn.InstanceName),
        "connection_type": connection_map.get(conn.VideoOutputTechnology,
                                              f"Unknown ({conn.VideoOutputTechnology})")
    }
    for conn in c_mon.WmiMonitorConnectionParams()
]

# --- Материнская плата ---
data["motherboard"] = [
    {
        "model": safe_str(board.Product),
        "serial": safe_str(board.SerialNumber),
        "manufacturer": safe_str(board.Manufacturer)
    }
    for board in c.Win32_BaseBoard()
]

# --- Процессоры ---
data["cpus"] = [
    {
        "name": safe_str(cpu.Name),
        "id": safe_str(cpu.ProcessorId),
        "socket": safe_str(cpu.SocketDesignation)
    }
    for cpu in c.Win32_Processor()
]

# --- Оперативная память ---
data["memory"] = [
    {
        "part_number": safe_str(mem.PartNumber),
        "serial": safe_str(mem.SerialNumber),
        "manufacturer": safe_str(mem.Manufacturer),
        "capacity_MB": int(mem.Capacity) // (1024*1024)
    }
    for mem in c.Win32_PhysicalMemory()
]

# --- Видеоустройства ---
data["gpus"] = [
    {
        "name": safe_str(gpu.Name),
        "device_id": safe_str(gpu.DeviceID),
        "pnp_id": safe_str(gpu.PNPDeviceID),
        "driver": safe_str(gpu.DriverVersion)
    }
    for gpu in c.Win32_VideoController()
]

# --- USB устройства ---
data["usb_devices"] = []
for usb in c.Win32_USBControllerDevice():
    try:
        dev = usb.Dependent
        data["usb_devices"].append({
            "name": safe_str(dev.Name),
            "device_id": safe_str(dev.DeviceID)
        })
    except Exception:
        pass

# --- Сетевые адаптеры ---
data["network_adapters"] = []
for nic in c.Win32_NetworkAdapterConfiguration(IPEnabled=True):
    data["network_adapters"].append({
        "description": safe_str(nic.Description),
        "mac": safe_str(nic.MACAddress),
        "ip_addresses": nic.IPAddress if nic.IPAddress else []
    })

# --- Экспорт в JSON ---
with open("system_info.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Информация сохранена в system_info.json")


Информация сохранена в system_info.json


In [25]:
def parse_edid(edid_bytes):
    manufacturer_id = ((edid_bytes[8] << 8) | edid_bytes[9])
    # 5 бит на букву
    mfg = "".join([
        chr(((manufacturer_id >> 10) & 0x1F) + 64),
        chr(((manufacturer_id >> 5) & 0x1F) + 64),
        chr((manufacturer_id & 0x1F) + 64)
    ])
    product_code = edid_bytes[10] | (edid_bytes[11] << 8)
    serial = int.from_bytes(edid_bytes[12:16], byteorder="little")
    week = edid_bytes[16]
    year = edid_bytes[17] + 1990
    return {
        "manufacturer": mfg,
        "product_code": product_code,
        "serial_number": serial,
        "manufacture_week": week,
        "manufacture_year": year
    }

# Использование:
for desc in c.WmiMonitorDescriptor():
    if desc.DescriptorType == 1:
        edid = bytes(desc.Descriptor)
        info = parse_edid(edid)
        print(info)


c:\Users\miwn36\AppData\Local\Programs\Python\Python312\Lib\site-packages\wmi.py:2: SyntaxWarning: invalid escape sequence '\_'
  """Windows Management Instrumentation (WMI) is Microsoft's answer to
c:\Users\miwn36\AppData\Local\Programs\Python\Python312\Lib\site-packages\wmi.py:917: SyntaxWarning: invalid escape sequence '\_'
  """This is the equivalent to the raw-WMI SpawnInstance\_


AttributeError: winmgmts:.WmiMonitorDescriptor

In [30]:
# inventory.py
import wmi
import json

def safe_str(val):
    return str(val).strip() if val else ""

def decode_wmi_string(arr):
    return "".join([chr(i) for i in arr if i > 0]).strip()

def collect_system_info():
    c = wmi.WMI()
    c_mon = wmi.WMI(namespace="root\\wmi")

    data = {}

    # --- Процессоры ---
    data["cpus"] = [
        {"name": safe_str(cpu.Name),
         "id": safe_str(cpu.ProcessorId),
         "socket": safe_str(cpu.SocketDesignation)}
        for cpu in c.Win32_Processor()
    ]

    # --- Материнская плата ---
    data["motherboard"] = [
        {"model": safe_str(board.Product),
         "serial": safe_str(board.SerialNumber),
         "manufacturer": safe_str(board.Manufacturer)}
        for board in c.Win32_BaseBoard()
    ]

    # --- BIOS ---
    data["bios"] = [
        {"manufacturer": safe_str(b.Manufacturer),
         "version": safe_str(b.SMBIOSBIOSVersion),
         "serial": safe_str(b.SerialNumber)}
        for b in c.Win32_BIOS()
    ]

    # --- Жёсткие диски ---
    data["disks"] = [
        {"model": safe_str(disk.Model),
         "serial": safe_str(disk.SerialNumber),
         "device_id": safe_str(disk.DeviceID),
         "size_bytes": int(disk.Size) if disk.Size else None}
        for disk in c.Win32_DiskDrive()
    ]

    # --- Оперативная память ---
    data["memory"] = [
        {"part_number": safe_str(mem.PartNumber),
         "serial": safe_str(mem.SerialNumber),
         "manufacturer": safe_str(mem.Manufacturer),
         "capacity_MB": int(mem.Capacity) // (1024*1024)}
        for mem in c.Win32_PhysicalMemory()
    ]

    # --- Видеокарты ---
    data["gpus"] = [
        {"name": safe_str(gpu.Name),
         "device_id": safe_str(gpu.DeviceID),
         "pnp_id": safe_str(gpu.PNPDeviceID),
         "driver": safe_str(gpu.DriverVersion)}
        for gpu in c.Win32_VideoController()
    ]

    # --- Мониторы ---
    data["monitors"] = [
        {"name": decode_wmi_string(m.UserFriendlyName),
         "manufacturer": decode_wmi_string(m.ManufacturerName),
         "serial": decode_wmi_string(m.SerialNumberID)}
        for m in c_mon.WmiMonitorID()
    ]

    # --- Подключение мониторов ---
    connection_map = {
        0: "Встроенный",
        1: "VGA",
        2: "S-Video",
        3: "Composite",
        4: "Component",
        5: "DVI/HDMI (TMDS)",
        6: "HDMI",
        8: "LVDS",
        10: "DisplayPort",
        11: "WirelessDisplay"
    }
    data["monitor_connections"] = [
        {"instance": safe_str(conn.InstanceName),
         "connection_type": connection_map.get(conn.VideoOutputTechnology,
                                               f"Unknown ({conn.VideoOutputTechnology})")}
        for conn in c_mon.WmiMonitorConnectionParams()
    ]

    # --- Сетевые адаптеры ---
    data["network_adapters"] = []
    for nic in c.Win32_NetworkAdapterConfiguration(IPEnabled=True):
        try:
            adapter = c.Win32_NetworkAdapter(DeviceID=nic.Index)[0]
            if adapter.Speed:
                speed_mbps = int(adapter.Speed) // 1_000_000
            else:
                speed_mbps = None
        except Exception:
            speed_mbps = None
        data["network_adapters"].append({
            "description": safe_str(nic.Description),
            "mac": safe_str(nic.MACAddress),
            "ip_addresses": nic.IPAddress if nic.IPAddress else [],
            "link_speed_mbps": speed_mbps
        })

    # --- USB устройства ---
    data["usb_devices"] = []
    for usb in c.Win32_USBControllerDevice():
        try:
            dev = usb.Dependent
            data["usb_devices"].append({
                "name": safe_str(dev.Name),
                "device_id": safe_str(dev.DeviceID)
            })
        except Exception:
            pass

    return data

if __name__ == "__main__":
    info = collect_system_info()
    with open("system_info.json", "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=4)
    print("Информация сохранена в system_info.json")


Информация сохранена в system_info.json


In [29]:
# full_inventory.py
import wmi
import json

def safe_str(val):
    return str(val).strip() if val else ""

def collect_full_info():
    c = wmi.WMI()
    data = {}

    # --- Процессоры ---
    data["processors"] = [
        {
            "name": safe_str(cpu.Name),
            "id": safe_str(cpu.ProcessorId),
            "manufacturer": safe_str(cpu.Manufacturer),
            "cores": cpu.NumberOfCores,
            "threads": cpu.NumberOfLogicalProcessors,
            "max_clock_mhz": cpu.MaxClockSpeed,
            "socket": safe_str(cpu.SocketDesignation)
        }
        for cpu in c.Win32_Processor()
    ]

    # --- Материнская плата ---
    data["baseboard"] = [
        {
            "manufacturer": safe_str(board.Manufacturer),
            "product": safe_str(board.Product),
            "serial": safe_str(board.SerialNumber),
            "version": safe_str(board.Version)
        }
        for board in c.Win32_BaseBoard()
    ]

    # --- BIOS ---
    data["bios"] = [
        {
            "manufacturer": safe_str(b.Manufacturer),
            "version": safe_str(b.SMBIOSBIOSVersion),
            "serial": safe_str(b.SerialNumber),
            "release_date": safe_str(b.ReleaseDate)
        }
        for b in c.Win32_BIOS()
    ]

    # --- Жёсткие диски ---
    data["disks"] = [
        {
            "model": safe_str(disk.Model),
            "serial": safe_str(disk.SerialNumber),
            "device_id": safe_str(disk.DeviceID),
            "interface": safe_str(disk.InterfaceType),
            "size_bytes": int(disk.Size) if disk.Size else None
        }
        for disk in c.Win32_DiskDrive()
    ]

    # --- Оперативная память ---
    data["memory_modules"] = [
        {
            "manufacturer": safe_str(mem.Manufacturer),
            "part_number": safe_str(mem.PartNumber),
            "serial": safe_str(mem.SerialNumber),
            "capacity_MB": int(mem.Capacity) // (1024*1024),
            "speed_MHz": mem.Speed
        }
        for mem in c.Win32_PhysicalMemory()
    ]

    # --- Видеокарты ---
    data["video_controllers"] = [
        {
            "name": safe_str(gpu.Name),
            "driver_version": safe_str(gpu.DriverVersion),
            "device_id": safe_str(gpu.DeviceID),
            "pnp_id": safe_str(gpu.PNPDeviceID),
            "adapter_ram_MB": int(gpu.AdapterRAM) // (1024*1024) if gpu.AdapterRAM else None
        }
        for gpu in c.Win32_VideoController()
    ]

    # --- Мониторы ---
    data["desktop_monitors"] = [
        {
            "name": safe_str(mon.Name),
            "device_id": safe_str(mon.DeviceID),
            "screen_height": mon.ScreenHeight,
            "screen_width": mon.ScreenWidth,
            "status": safe_str(mon.Status)
        }
        for mon in c.Win32_DesktopMonitor()
    ]

    # --- Сетевые адаптеры ---
    data["network_adapters"] = [
        {
            "name": safe_str(nic.Name),
            "description": safe_str(nic.Description),
            "mac": safe_str(nic.MACAddress),
            "adapter_type": safe_str(nic.AdapterType),
            "speed_mbps": int(nic.Speed) // 1_000_000 if nic.Speed else None,
            "status": safe_str(nic.NetConnectionStatus)
        }
        for nic in c.Win32_NetworkAdapter() if nic.PhysicalAdapter
    ]

    # --- Звуковые устройства ---
    data["sound_devices"] = [
        {
            "name": safe_str(snd.Name),
            "manufacturer": safe_str(snd.Manufacturer),
            "device_id": safe_str(snd.DeviceID),
            "status": safe_str(snd.Status)
        }
        for snd in c.Win32_SoundDevice()
    ]

    return data


if __name__ == "__main__":
    info = collect_full_info()
    with open("system_info.json", "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=4)
    print("Информация сохранена в system_info.json")


Информация сохранена в system_info.json
